## Clone out interfacegan

In [ ]:
# Kaggle
!git clone https://github.com/ficinator/interfacegan.git && cd interfacegan && git checkout kaggle

In [ ]:
import sys
import logging
import numpy as np
from glob import glob
from pathlib import Path
from collections import defaultdict
from tqdm.notebook import tqdm
from PIL import Image

from interfacegan.models.stylegan_generator import StyleGANGenerator

In [ ]:
# set up logging
logger = logging.getLogger('generate_data')

if not logger.handlers:
    sh = logging.StreamHandler(sys.stdout)
    sh.setFormatter(logging.Formatter(
        fmt='%(asctime)s [%(levelname)s] %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
    ))
    logger.addHandler(sh)
    
logger.setLevel(logging.DEBUG)

In [ ]:
# Colab
# from google.colab import drive
# drive.mount('/content/drive')

## Download the Pretrained Model

In [ ]:
model_name = 'stylegan_ffhq'
model_path = Path('interfacegan/models/pretrain')/f'{model_name}.pth'

if not model_path.exists():
    !wget https://www.dropbox.com/s/qyv37eaobnow7fu/{model_name}.pth?dl=1 -O {model_path}
model = StyleGANGenerator(model_name)

## Generate Images

To Each run generate only 10k images to fit into Kaggle limits. It takes around 15 minutes and the final zip file has around 3 GB.

In [ ]:
run_num = 0
num_images = 10000
offset = run_num * num_images

output_dir = Path('output')/model_name
images_dir = output_dir/'images'
latents_dir = output_dir/'latents'

images_dir.mkdir(parents=True, exist_ok=True)
for latent_space_type in ['w', 'wp', 'z']:
    (latents_dir/latent_space_type).mkdir(parents=True, exist_ok=True)

# !python interfacegan/generate_data.py -m {model_name} -o {tmp_dir} -n {num_images}

logger.info(f'Run {run_num}: started generating {num_images} images from {latent_space_type} latents')

latent_codes = model.easy_sample(num_images)
pbar = tqdm(total=num_images)
batch_size = model.batch_size

for batch_num, latent_codes_batch in enumerate(model.get_batch_inputs(latent_codes)):
    outputs = model.easy_synthesize(latent_codes_batch)
    i = batch_num * batch_size
    for key, val in outputs.items():
        if key == 'image':
            for j, image in enumerate(val):
                Image.fromarray(image).save(images_dir/f'{i + j + offset:06d}.jpg')
        else:
            for j, latent in enumerate(val):
                np.save(latents_dir/key/f'{i + j + offset:06d}.npy', latent)
    pbar.update(len(latent_codes_batch))
pbar.close()

logger.info(f'Run {run_num}: finished generating {num_images} images from {latent_space_type} latents')

## Zip the Images and dlatents

In [ ]:
zip_path = f'{model_name}_{run_num:02d}.zip'
logger.info(f'Started compressing data to {zip_path}')
!zip -qr {zip_path} {output_dir}
logger.info(f'Finished compressing data to {zip_path}')

## Combine Data

After unziping all the folders and moving all the files to `data/stylegan_ffhq` folder run the following cell to concatenate the dlatents

In [ ]:
data_dir = Path('data')/model_name

In [ ]:
for latent_space_type in ['w', 'wp', 'z']:
    paths = sorted(glob(str(DATA_DIR/f'{latent_space_type}_*.npy')))
    dlatents = np.vstack(list(map(np.load, paths)))
    np.save(DATA_DIR/f'{latent_space_type}.npy', dlatents)

In [ ]:
dlatents_dir = data_dir/'dlatents'

for latents_path in dlatents_dir.glob('*.npy'):
    latent_space_type = latents_path.stem
    latents_dir = data_dir/'latents'/latent_space_type
    latents_dir.mkdir(parents=True, exist_ok=True)
    latents = np.load(latents_path)
    logger.debug(latents.shape)
    for i, latent in enumerate(latents):
        np.save(latents_dir/f'{i + 10000:06d}.npy', latent)